In [2]:
using Plots,Turing, Distributions
include("../rpc.jl")

latestrichblocks (generic function with 1 method)

In [2]:
ipc = connect("/home/keorn/.local/share/io.parity.ethereum/jsonrpc.ipc")

Base.PipeEndpoint(open, 0 bytes waiting)

In [3]:
blocks = latestblocks(ipc, 1000);

In [4]:
NH = 1150000
D0 = 131072
function newdifficulty(parent, number, timediff)
    x = floor(parent["difficulty"]/2048)
    ϵ = floor(2^(floor(number/100000) - 2))
    ζ = if number < NH
         timediff < 13 ? 1 : -1
        else
         max(1 - floor((timediff)/10), -99)
    end
    max(D0, parent["difficulty"] + x* ζ + ϵ)
end

newdifficulty (generic function with 1 method)

In [5]:
function newgaslimit(parentgaslimit)
    maxchange = floor(parentgaslimit / 1024)
    DiscreteUniform(max(125000, parentgaslimit - maxchange), parentgaslimit + maxchange)
end

newgaslimit (generic function with 1 method)

In [6]:
abstract StringDistribution <: DiscreteUnivariateDistribution

In [7]:
immutable HashString <: StringDistribution
    bit::UInt
end

In [8]:
Distributions.rand(d::HashString) = string([hex(rand(0x0:0xF)) for _ in 1:div(d.bit, 4)]...)
function Distributions.logpdf(d::HashString, x::String)
    if ismatch(r"^[0-9,a-f]+$", x) && length(x) * 4 == d.bit
        -Int(d.bit)*log(2)
    else
        -Inf
    end
end

In [9]:
Distributions.rand(d::HashString, n::Int) = Vector([rand(d) for _ in 1:n])
Distributions.logpdf(d::HashString, xs::Vector{String}) = sum(logpdf(d, x) for x in xs)

In [10]:
immutable VectorDistribution <: DiscreteUnivariateDistribution
    inner::Distribution
    maxelements::Int
end

In [11]:
Distributions.rand(d::VectorDistribution) = rand(d.inner, rand(0:d.maxelements))
function Distributions.logpdf(d::VectorDistribution, v::Vector)
    l = length(x)
    logpdf(DiscreteUniform(1, d.maxelements), l) + sum(logpdf(d.inner, i) for i in v)
end
Distributions.rand(d::VectorDistribution, n::Int) = Vector([rand(d) for _ in 1:n])
Distributions.logpdf(d::VectorDistribution, xs::Vector{Vector}) = sum(logpdf(d, x) for x in xs)

In [12]:
immutable DataString <: StringDistribution
    maxbytes::UInt
end
Distributions.rand(d::DataString) = rand(HashString(rand(8:8*d.maxbytes)))
function Distributions.logpdf(d::DataString, x::String)
    l = length(x)
    logpdf(DiscreteUniform(1, d.maxbytes), div(l, 2)) + logpdf(HashString(l * 4), x)
end
Distributions.rand(d::DataString, n::Int) = Vector([rand(d) for _ in 1:n])
Distributions.logpdf(d::DataString, xs::Vector{String}) = sum(logpdf(d, x) for x in xs)

In [13]:
Kronecker(a) = DiscreteUniform(a, a)
immutable PrefixedString <: StringDistribution
    prefix::String
    inner::StringDistribution
end
Distributions.rand(d::PrefixedString) = d.prefix * rand(d.inner)
function Distributions.logpdf(d::PrefixedString, x::String)
    matches = match(Regex("^" * d.prefix * "(.*)\$"), x)
    matches === nothing ? -Inf : logpdf(d.inner, matches.captures[1])
end
Distributions.rand(d::PrefixedString, n::Int) = Vector([rand(d) for _ in 1:n])
Distributions.logpdf(d::PrefixedString, xs::Vector{String}) = sum(logpdf(d, x) for x in xs)

In [12]:
function blockdistributions(history)
    number = history[end]["number"] + 1
    author = PrefixedString("0x", HashString(160))
    timediff = rand(Poisson(14))
    difficulty = newdifficulty(history[end], number, timediff)
    gaslimit = newgaslimit(history[end]["gasLimit"])
    gasused = DiscreteUniform(0, rand(gaslimit))
    mixhash = HashString(256)
    nonce = HashString(64)

    Dict(
    "totalDifficulty" => Kronecker(history[end]["totalDifficulty"] + difficulty),
    "extraData" => PrefixedString("0x", DataString(32)),
    "number" => Kronecker(number),
    "difficulty" => Kronecker(difficulty),
    "miner" => author,
    "logsBloom" => Kronecker(0),
    "transactionsRoot" => PrefixedString("0x", HashString(256)),
    "author" => author,
    "sealFields" => [PrefixedString("0xa0", mixhash), PrefixedString("0x88", nonce)],
    "sha3Uncles" => PrefixedString("0x", HashString(256)),
    "parentHash" => Kronecker(history[end-1]["hash"]),
    "gasLimit" => gaslimit,
    "hash" => PrefixedString("0x", HashString(256)),
    "size" => DiscreteUniform(0, 100000),
    "mixHash" => PrefixedString("0x", mixhash),
    "receiptsRoot" => PrefixedString("0x", HashString(256)),
    "stateRoot" => PrefixedString("0x", HashString(256)),
    "gasUsed" => gasused,
    "transactions" => VectorDistribution(PrefixedString("0x", HashString(256)), div(rand(gasused), rand(DiscreteUniform(21000, rand(gasused))))),
    "timestamp" => history[end]["timestamp"] + timediff,
    "uncles" => VectorDistribution(PrefixedString("0x", HashString(256)), 2),
    "nonce" => PrefixedString("0x", nonce)
    )
end

blockdistributions (generic function with 1 method)

In [28]:
blockdistributions(blocks)

LoadError: MethodError: no method matching Distributions.DiscreteUniform(::String, ::String)[0m
Closest candidates are:
  Distributions.DiscreteUniform{T}(::Any) at sysimg.jl:53[0m

In [14]:
@model block(history) = begin
    number = history[end]["number"] + 1
    author ~ PrefixedString("0x", HashString(160))
    timediff ~ Poisson(14)
    difficulty = newdifficulty(history[end], number, timediff)
    gaslimit ~ newgaslimit(history[end]["gasLimit"])
    gasused ~ DiscreteUniform(0, gaslimit)
    mixhash ~ HashString(256)
    nonce ~ HashString(64)
    extradata ~ PrefixedString("0x", DataString(32))
    transactionsroot ~ PrefixedString("0x", HashString(256))
    sha3uncles ~ PrefixedString("0x", HashString(256))
    hash ~ PrefixedString("0x", HashString(256))
    size ~ DiscreteUniform(0, 100000)
    receiptsroot ~ PrefixedString("0x", HashString(256))
    stateroot ~ PrefixedString("0x", HashString(256))
    averagetxgas ~ DiscreteUniform(21000, max(21000, gasused))
    transactions ~ VectorDistribution(PrefixedString("0x", HashString(256)), div(gasused, averagetxgas))
    uncles ~ VectorDistribution(PrefixedString("0x", HashString(256)), 2)

    Dict(
    "totalDifficulty" => history[end]["totalDifficulty"] + difficulty,
    "extraData" => extradata,
    "number" => number,
    "difficulty" => difficulty,
    "miner" => author,
    "logsBloom" => 0,
    "transactionsRoot" => transactionsroot,
    "author" => author,
    "sealFields" => [string("0xa0", mixhash), string("0x88", nonce)],
    "sha3Uncles" => sha3uncles,
    "parentHash" => history[end-1]["hash"],
    "gasLimit" => gaslimit,
    "hash" => hash,
    "size" => size,
    "mixHash" => string("0x", mixhash),
    "receiptsRoot" => receiptsroot,
    "stateRoot" => stateroot,
    "gasUsed" => gasused,
    "transactions" => transactions,
    "timestamp" => history[end]["timestamp"] + timediff,
    "uncles" => uncles,
    "nonce" => string("0x", nonce)
    )
end

block (generic function with 2 methods)

In [18]:
c1 = sample(block(blocks), LMH(10))

[Turing]: Assume - `author` is a parameter
[Turing]: Assume - `timediff` is a parameter
[Turing]: Assume - `gaslimit` is a parameter
[Turing]: Assume - `gasused` is a parameter
[Turing]: Assume - `mixhash` is a parameter
[Turing]: Assume - `nonce` is a parameter
[Turing]: Assume - `extradata` is a parameter
[Turing]: Assume - `transactionsroot` is a parameter
[Turing]: Assume - `sha3uncles` is a parameter
[Turing]: Assume - `hash` is a parameter (ignoring `hash` found in global scope)
[Turing]: Assume - `size` is a parameter (ignoring `size` found in global scope)
[Turing]: Assume - `receiptsroot` is a parameter
[Turing]: Assume - `stateroot` is a parameter
[Turing]: Assume - `averagetxgas` is a parameter
[Turing]: Assume - `transactions` is a parameter
[Turing]: Assume - `uncles` is a parameter


LoadError: UndefVarError: LMH not defined

In [22]:
[1, 2][length([1, 2])]

2

In [20]:
c1

Object of type "Turing.Chain"

Iterations = 1:10
Thinning interval = 1
Chains = 1
Samples per chain = 10

[0.0 6.79645e18 0.0; 0.0 6.33299e18 0.0; … ; 0.0 1.47219e18 0.0; 0.0 3.83103e18 0.0]

In [17]:
"a" * "b"

"ab"

In [23]:
timestamps = [b["timestamp"] for b in blocks]
times = [times[i]-times[i-1] for i in 2:length(times)]
histogram(times)

In [27]:
histogram([rand(Poisson(14)) for _ in 1:999])

In [84]:
[b["size"] for b in blocks]

1000-element Array{Int64,1}:
  544
  549
  544
  547
  544
  689
  544
  547
  660
  547
  549
  522
 1856
    ⋮
  773
  517
  544
 1085
  720
  663
  549
  517
  547
  517
  663
  544

In [17]:
histogram([length(b["extraData"]) for b in blocks])

In [9]:
Plots.plot(t->24921 / (t + 1.56) + 25, 0, 24)

In [8]:
Plots.plot(t->25600/(t+1.6) - 25, 0, 2000)